In [1]:
# Let's first install Cohere's python SDK
# %pip install cohere
# %pip install python-dotenv

In [2]:
import cohere
import pandas as pd
import requests
import datetime
from tqdm import tqdm
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)

import sys,os
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from logger import Logger
from dotenv import load_dotenv

In [3]:
load_dotenv()
# Paste your API key here. Remember to not share publicly
api_key = os.getenv('COHERE_API_KEY')

# log 
logger = Logger("../logs/jd_entity_extraction.log").get_app_logger()

In [4]:
# load the train dataset
try:
    df_train = pd.read_json('../data/relations_dev.json')
    logger.info('load data successfully')
except:
    logger.exception('failed to load dataset')

In [5]:
df_train.head()

,document,tokens,relations
0,"Bachelor's degree in Mechanical Engineering or Physical Science 3+ years track record of developing or specifying fiber optic cables and connector related products Knowledge of fiber optic component, cabling, and interconnect products, technologies, and standards Experience in statistical data analysis Experience with product life cycle management (PLM) process Experience providing solutions to problems and meeting deadlines Experience engaging stakeholders PREFERRED Advanced degree Experience using a software tool for statistical data analysis such as JMP Experience using Agile as product life-cycle management tool Data center or other mission critical development experience","[{'text': 'Bachelor', 'start': 0, 'end': 8, 'token_start': 0, 'token_end': 0, 'entityLabel': 'DIPLOMA'}, {'text': 'Mechanical Engineering', 'start': 21, 'end': 43, 'token_start': 4, 'token_end': 5, 'entityLabel': 'DIPLOMA_MAJOR'}, {'text': 'Physical Science', 'start': 47, 'end': 63, 'token_start': 7, 'token_end': 8, 'entityLabel': 'DIPLOMA_MAJOR'}, {'text': '3+ years', 'start': 64, 'end': 72, 'token_start': 9, 'token_end': 11, 'entityLabel': 'EXPERIENCE'}, {'text': 'developing', 'start': 89, 'end': 99, 'token_start': 15, 'token_end': 15, 'entityLabel': 'SKILLS'}, {'text': 'fiber optic cables', 'start': 114, 'end': 132, 'token_start': 18, 'token_end': 20, 'entityLabel': 'SKILLS'}, {'text': 'connector related products', 'start': 137, 'end': 163, 'token_start': 22, 'token_end': 24, 'entityLabel': 'SKILLS'}]","[{'child': 4, 'head': 0, 'relationLabel': 'DEGREE_IN'}, {'child': 7, 'head': 0, 'relationLabel': 'DEGREE_IN'}, {'child': 15, 'head': 9, 'relationLabel': 'EXPERIENCE_IN'}, {'child': 18, 'head': 9, 'relationLabel': 'EXPERIENCE_IN'}, {'child': 22, 'head': 9, 'relationLabel': 'EXPERIENCE_IN'}]"
1,"10+ years of software engineering work experience. Technical experience in release automation engineering, CI/CD or related roles. Experience building and leading a software organization through product design, delivery and commercialization of consumer electronics devices. Experience recruiting and managing technical teams, including performance management. BS/MS in Computer Science. Experience in leading timeline, multi-partner initiatives. Organizational communication and coordination experience. PREFERRED 5+ years of experience with hands-on technical management, release engineering, tools engineering, DevOps, or related area.","[{'text': '10+ years', 'start': 0, 'end': 9, 'token_start': 0, 'token_end': 2, 'entityLabel': 'EXPERIENCE'}, {'text': 'software engineering', 'start': 13, 'end': 33, 'token_start': 4, 'token_end': 5, 'entityLabel': 'SKILLS'}, {'text': '5+ years', 'start': 515, 'end': 523, 'token_start': 77, 'token_end': 79, 'entityLabel': 'EXPERIENCE'}, {'text': 'technical management', 'start': 552, 'end': 572, 'token_start': 86, 'token_end': 87, 'entityLabel': 'SKILLS'}, {'text': 'release engineering', 'start': 574, 'end': 593, 'token_start': 89, 'token_end': 90, 'entityLabel': 'SKILLS'}, {'text': 'tools engineering', 'start': 595, 'end': 612, 'token_start': 92, 'token_end': 93, 'entityLabel': 'SKILLS'}, {'text': 'DevOps', 'start': 614, 'end': 620, 'token_start': 95, 'token_end': 95, 'entityLabel': 'SKILLS'}, {'text': 'BS/MS', 'start': 361, 'end': 366, 'token_start': 53, 'token_end': 55, 'entityLabel': 'DIPLOMA'}, {'text': 'Computer Science', 'start': 370, 'end': 386, 'token_start': 57, 'token_end': 58, 'entityLabel': 'DIPLOMA_MAJOR'}]","[{'child': 4, 'head': 0, 'relationLabel': 'EXPERIENCE_IN'}, {'child': 86, 'head': 77, 'relationLabel': 'EXPERIENCE_IN'}, {'child': 89, 'head': 77, 'relationLabel': 'EXPERIENCE_IN'}, {'child': 92, 'head': 77, 'relationLabel': 'EXPERIENCE_IN'}, {'child': 95, 'head': 77, 'relationLabel': 'EXPERIENCE_IN'}, {'child': 57, 'head': 53, 'relationLabel': 'DEGREE_IN'}]"
2,"3+ years Swift & Objective-C and experience with iOS internals Experience building an entire app from scratch and ideally a po

In [6]:
# load the train dataset
try:
    df_test= pd.read_json('../data/relations_test.json')
    logger.info('load data successfully')
except:
    logger.exception('failed to load dataset')

In [7]:
# Let's glance at the dataset
df_test.head()

,document,tokens,relations
0,"\nCurrently holding a faculty, industry, or government researcher position.\nPh.D. and publications in machine learning, AI, computer science, statistics, applied mathematics, data science, or related technical fields.\nExperience leading a team in solving analytical problems using quantitative approaches.\nExperience manipulating and analyzing data from different sources.\nExperience in theoretical and empirical research and for answering questions with research.\nAbility to communicate research for public audiences of peers.\nKnowledge in a programming language.\nAbility to obtain and maintain work authorization in the country of employment in 2018.\n\nPREFERRED \n1+ year(s) of work experience in a university, industry, or government lab(s), in a role with primary emphasis on AI research.\nExperience driving original scholarship in collaboration with a team.\nFirst-author publications at peer-reviewed AI conferences (e.g. NIPS, CVPR, ICML, ICLR, ICCV, and ACL).\nExperience in developing and debugging in C/C++, Python, C# and/or Java.","[{'text': 'Ph.D.', 'start': 75, 'end': 80, 'token_start': 14, 'token_end': 14, 'entityLabel': 'DIPLOMA'}, {'text': 'machine learning', 'start': 101, 'end': 117, 'token_start': 18, 'token_end': 19, 'entityLabel': 'DIPLOMA_MAJOR'}, {'text': 'AI', 'start': 119, 'end': 121, 'token_start': 21, 'token_end': 21, 'entityLabel': 'DIPLOMA_MAJOR'}, {'text': 'computer science', 'start': 123, 'end': 139, 'token_start': 23, 'token_end': 24, 'entityLabel': 'DIPLOMA_MAJOR'}, {'text': 'statistics', 'start': 141, 'end': 151, 'token_start': 26, 'token_end': 26, 'entityLabel': 'DIPLOMA_MAJOR'}, {'text': 'applied mathematics', 'start': 153, 'end': 172, 'token_start': 28, 'token_end': 29, 'entityLabel': 'DIPLOMA_MAJOR'}, {'text': 'data science', 'start': 174, 'end': 186, 'token_start': 31, 'token_end': 32, 'entityLabel': 'DIPLOMA_MAJOR'}, {'text': '1+ year(s', 'start': 664, 'end': 673, 'token_start': 113, 'token_end': 115, 'entityLabel': 'EXPERIENCE'}, {'text': 'university', 'start': 699, 'end': 709, 'token_start': 122, 'token_end': 122, 'entityLabel': 'SKILLS'}, {'text': 'industry', 'start': 711, 'end': 719, 'token_start': 124, 'token_end': 124, 'entityLabel': 'SKILLS'}, {'text': 'government lab(s)', 'start': 724, 'end': 741, 'token_start': 127, 'token_end': 129, 'entityLabel': 'SKILLS'}, {'text': 'AI', 'start': 778, 'end': 780, 'token_start': 138, 'token_end': 138, 'entityLabel': 'SKILLS'}]","[{'child': 18, 'head': 14, 'relationLabel': 'DEGREE_IN'}, {'child': 21, 'head': 14, 'relationLabel': 'DEGREE_IN'}, {'child': 23, 'head': 14, 'relationLabel': 'DEGREE_IN'}, {'child': 26, 'head': 14, 'relationLabel': 'DEGREE_IN'}, {'child': 28, 'head': 14, 'relationLabel': 'DEGREE_IN'}, {'child': 31, 'head': 14, 'relationLabel': 'DEGREE_IN'}, {'child': 122, 'head': 113, 'relationLabel': 'EXPERIENCE_IN'}, {'child': 124, 'head': 113, 'relationLabel': 'EXPERIENCE_IN'}, {'child': 127, 'head': 113, 'relationLabel': 'EXPERIENCE_IN'}, {'child': 138, 'head': 113, 'relationLabel': 'EXPERIENCE_IN'}]"
1,"\n2+ years experience in the online advertising or research\nBS/BA Degree in Economics, Statistics, Political Science, History, Psychology\nExperience with presenting and partnering with technical and non-technical teams\nExperience communicating analyses and results to any audience\nExperience working with structured and unstructured data-sets, statistical software such as R, STATA, SPSS, SAS as well as data extraction tools such as Hive and/or SQL\n\nPREFERRED \nExperience with causal measurement, machine learning or lab-based research\nExperience in quantitative field, such as consulting, market research, strategy and planning, or user experience research\nStrong organizational and project management skills\nMasters in quantitative field or a MBA","[{'text': '2+ years', 'start': 1, 'end': 9, 'token_start': 1, 'token_end': 3, 'entityLabel': 'EXPERIENCE'}, {'text': 'online advertising', 'start': 28, 'end': 

In [8]:
from preprocessing import Preporcess
preprocess = Preporcess() 

# preprocess the tokens for both the training and testing dataset
df_train['label'] = df_train['tokens'].apply(preprocess.preprocess_tokens)
df_test['label'] = df_test['tokens'].apply(preprocess.preprocess_tokens)

In [9]:
df_train['label']

0                                                                                            DIPLOMA:Bachelor\nDIPLOMA_MAJOR:Mechanical Engineering,Physical Science\nEXPERIENCE:3+ years\nSKILLS:developing,fiber optic cables,connector related products\n
1                                                                              EXPERIENCE:10+ years,5+ years\nSKILLS:software engineering,technical management,release engineering,tools engineering,DevOps\nDIPLOMA:BS/MS\nDIPLOMA_MAJOR:Computer Science\n
2                                                                                                                                                                                                          EXPERIENCE:3+ years\nSKILLS:Swift & Objective-C\n
3                                                                                                                                                             EXPERIENCE:8+ years,5+ years\nSKILLS:software engineering,people management,managin

In [10]:
train_data = preprocess.preprocess_document(df_train)
train_data

["Bachelor's degree in Mechanical Engineering or Physical Science 3+ years track record of developing or specifying fiber optic cables and connector related products Knowledge of fiber optic component, cabling, and interconnect products, technologies, and standards Experience in statistical data analysis Experience with product life cycle management (PLM) process Experience providing solutions to problems and meeting deadlines Experience engaging stakeholders PREFERRED Advanced degree Experience using a software tool for statistical data analysis such as JMP Experience using Agile as product life-cycle management tool Data center or other mission critical development experience\n\nExtracted Text:\nDIPLOMA:Bachelor\nDIPLOMA_MAJOR:Mechanical Engineering,Physical Science\nEXPERIENCE:3+ years\nSKILLS:developing,fiber optic cables,connector related products\n----\n",
 '10+ years of software engineering work experience. Technical experience in release automation engineering, CI/CD or related

In [11]:
test_data = preprocess.entity_extraction(df_test)
test_data

[' Currently holding a faculty, industry, or government researcher position. Ph.D. and publications in machine learning, AI, computer science, statistics, applied mathematics, data science, or related technical fields. Experience leading a team in solving analytical problems using quantitative approaches. Experience manipulating and analyzing data from different sources. Experience in theoretical and empirical research and for answering questions with research. Ability to communicate research for public audiences of peers. Knowledge in a programming language. Ability to obtain and maintain work authorization in the country of employment in 2018.  PREFERRED  1+ year(s) of work experience in a university, industry, or government lab(s), in a role with primary emphasis on AI research. Experience driving original scholarship in collaboration with a team. First-author publications at peer-reviewed AI conferences (e.g. NIPS, CVPR, ICML, ICLR, ICCV, and ACL). Experience in developing and debu

In [ ]:
coh = cohere.Client(api_key)